In [1]:
import json

In [2]:
import json

# Ruta al archivo (ajústala si está en otro directorio)
json_path = "dev-v2.0.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Verifica el contenido cargado
print(type(data))
print(list(data.keys()))


<class 'dict'>
['version', 'data']


In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [4]:
print(es_client.info())

{'name': 'ebdb662bb72e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QXs9K_-iSr2iyYqjpYEKkA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [5]:
titles = [item["title"] for item in data["data"]]

for t in titles:
    print(t)

Normans
Computational_complexity_theory
Southern_California
Sky_(United_Kingdom)
Victoria_(Australia)
Huguenot
Steam_engine
Oxygen
1973_oil_crisis
European_Union_law
Amazon_rainforest
Ctenophora
Fresno,_California
Packet_switching
Black_Death
Geology
Pharmacy
Civil_disobedience
Construction
Private_school
Harvard_University
Jacksonville,_Florida
Economic_inequality
University_of_Chicago
Yuan_dynasty
Immune_system
Intergovernmental_Panel_on_Climate_Change
Prime_number
Rhine
Scottish_Parliament
Islamism
Imperialism
Warsaw
French_and_Indian_War
Force


In [6]:
documents = []

for article in data["data"]:  # cada tema (con título)
    title = article["title"]
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            doc = {
                "title": title,
                "context": context,
                "question": qa["question"],
                "answers": [ans["text"] for ans in qa.get("answers", [])],
                #"is_impossible": qa.get("is_impossible", False),
                #"id": qa["id"]
            }
            documents.append(doc)


In [7]:
#print(len(documents))
documents[0]



{'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': ['France', 'France', 'France', 'France']}

In [10]:
print(es_client.indices.get_alias(index="*").keys())


dict_keys(['questions', 'course-questions'])


In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answers": {"type": "text"},
            "question": {"type": "text"},
            "title": {"type": "keyword"} 
        }
    }
}

index_name ="questions2"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions2'})

In [13]:
from tqdm.auto import tqdm 

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/11873 [00:00<?, ?it/s]

In [15]:
query = "Where is located Normandy?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "answers"],
                    "type": "best_fields"
                }
            },
        }
    }
}

search_results = es_client.search(index=index_name, body=search_query)

In [19]:
#search_results['hits']['hits'][0]['_score']
search_results['hits']['hits'][0]

{'_index': 'questions2',
 '_id': 'de_vqZcBHWPbPBrBn1oY',
 '_score': 62.14129,
 '_ignored': ['context.keyword'],
 '_source': {'title': 'Normans',
  'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
  'question': 'In what country is Normandy located?',
  'answers': ['France', 'France', 'France'

In [20]:
answers = search_results['hits']['hits'][0]['_source']['answers']
unique_answers = list(set(answers))
print("Answers:", ", ".join(unique_answers))


Answers: France


In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv() 

In [ ]:
client = OpenAI()

In [ ]:
def search(query):
    boost = {'question': 3.0}
    #Lucine elastic search
    results = index.search(
        query=query,
        filter_dict={'title': 'Normans'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [ ]:
query = 'Where is located Normandy?'
search_results = search(query)